In [1]:
from src.experiment_based_function import SEEDS
import pandas as pd
%matplotlib inline

In [2]:
import os
import glob


In [3]:
eval_or_grid='eval'

In [4]:
def get_exp8_result(eval_or_grid='eval'):
    res = []    
    for fn in glob.glob('experiment_1001/exp8-spatial-thres-i-up-fsel/*/*/%s_res.csv' % eval_or_grid):
        _, i_thres, seed, _ = fn.split(os.path.sep)
        seed = seed.replace('seed_','')
        df = pd.read_csv(fn, index_col=0)
        df['seed'] = seed
        df['i_thres']=i_thres
        res.append(df)
    return pd.concat(res)


In [5]:
eval_res = get_exp8_result('eval')

In [6]:
col_mapping = {' ': ' ',
 'RoadNet': 'RoadNet+Spatial',
 'Segment_NO_TOTAL_~2014+Spatial': 'Social type+Spatial', 
 'Segment_TOTAL_~2014+Spatial': 'Social total+Spatial',
 'RoadNet+Segment_NO_TOTAL_~2014+Spatial': 'RoadNet+Social type+Spatial', 
 'RoadNet+Segment_TOTAL_~2014+Spatial': 'RoadNet+Social total+Spatial'}


In [8]:
eval_res.ftr_combo_name = eval_res.ftr_combo_name.apply(lambda x: col_mapping[x.split('-')[0]] if x.split('-')[0] in col_mapping else x.split('-')[0])

In [9]:
eval_res.columns, eval_res.shape

(Index(['#ftr_all', '#ftr_keep', '#test_sample', '#train_sample',
        'feature_selection', 'ftr_combo_name', 'model_name', 'test_acc',
        'test_f1_macro', 'test_f1_micro', 'test_f1_weighted', 'test_mse',
        'test_n_classes', 'test_precision_macro', 'test_precision_micro',
        'test_precision_weighted', 'test_recall_macro', 'test_recall_micro',
        'test_recall_weighted', 'train_acc', 'train_f1_macro', 'train_f1_micro',
        'train_f1_weighted', 'train_mse', 'train_n_classes',
        'train_precision_macro', 'train_precision_micro',
        'train_precision_weighted', 'train_recall_macro', 'train_recall_micro',
        'train_recall_weighted', 'upsample', 'y_dist', 'y_dist_up', 'seed',
        'i_thres'],
       dtype='object'), (394, 36))

In [10]:
print('train', eval_res.train_n_classes.value_counts().to_dict())
print('test', eval_res.test_n_classes.value_counts().to_dict())


print('train', 'test')
print(eval_res.apply(lambda x: (x.train_n_classes, x.test_n_classes), axis=1).value_counts())

train {5: 394}
test {5: 278, 4: 116}
train test
(5, 5)    278
(5, 4)    116
dtype: int64


In [11]:
f1_choices = ['test_f1_weighted', 'test_f1_macro', 'test_f1_micro']

In [17]:
keys = ['model_name', 'i_thres', 'ftr_combo_name', 'feature_selection', 'upsample']
means = eval_res.groupby(keys).mean()[f1_choices+['#ftr_keep', '#ftr_all']]
means.test_f1_micro.max()

0.65617977528089888

In [13]:
means

test_f1_weighted  \
model_name i_thres ftr_combo_name               feature_selection upsample                     
BAGcls     0.0     RoadNet+Social total+Spatial mrmr              svm               0.609502   
                                                rfecv_linsvc      None              0.532469   
                                                                  svm               0.606289   
                   RoadNet+Social type+Spatial  mrmr              svm               0.594151   
                                                rfecv_linsvc      None              0.569363   
                                                                  svm               0.597272   
                   RoadNet+Spatial              rfecv_linsvc      None              0.573807   
                                                                  svm               0.620435   
                   Social total+Spatial         mrmr              svm               0.517415   
                                                rfecv_linsvc      None              0.473716   
                                                                  svm               0.551536   
                   Social type+Spatial          mrmr              svm               0.582351   
                                                rfecv_linsvc      None              0.559199   
                                                                  svm               0.589711   
           0.68    RoadNet+Social total+Spatial mrmr              regular           0.610756   
                                                rfecv_linsvc      None              0.568987   
                                                                  regular           0.616931   
                                                                  svm               0.623912   
                   RoadNet+Social type+Spatial  mrmr              svm               0.620994   
                                                rfecv_linsvc      None              0.570058   
                                                                  regular           0.591668   
                                                                  svm               0.619798   
                   RoadNet+Spatial              rfecv_linsvc      None              0.566943   
                                                                  regular           0.615309   
                                                                  svm               0.622609   
                   Social total+Spatial         mrmr              regular           0.504524   
                                                                  svm               0.493024   
                                                rfecv_linsvc      None              0.463573   
                                                                  regular           0.508059   
                                                                  svm               0.541279   
...                                                                                      ...   
XGBcls     0.0     RoadNet+Social type+Spatial  rfecv_linsvc      svm               0.611294   
                   RoadNet+Spatial              rfecv_linsvc      None              0.580627   
                                                                  svm               0.611780   
                   Social total+Spatial         mrmr              svm               0.537966   
                                                rfecv_linsvc      None              0.472077   
                                                                  svm               0.513597   
                   Social type+Spatial          mrmr              svm               0.638232   
                                                rfecv_linsvc      None              0.541616   
                                                                  svm               0.578202   
           0.68    RoadNet+Social total+Spat

In [32]:
from IPython.display import display

In [14]:
def get_paper_table(means, f1_type, key_cols):

    means_for_pivot = means.reset_index()
    means_for_pivot['key'] = means_for_pivot.apply(lambda x: tuple([x[col] for col in key_cols]), axis=1)
    table = means_for_pivot.reset_index().pivot(index='key', columns='ftr_combo_name', values=f1_type)
    table.index = pd.MultiIndex.from_tuples(table.index, names=key_cols)    
    table.columns.name = f1_type
    table = table[['RoadNet+Spatial', 
                   'Social total+Spatial',
                   'Social type+Spatial',
                   'RoadNet+Social total+Spatial',
                   'RoadNet+Social type+Spatial']]
    table.index.name=None
    table.columns.name=f1_type
    return table

In [23]:
slice_i = means.iloc[means.index.get_level_values(1) == '0.68']
slice_i.test_f1_macro.max(), slice_i.test_f1_micro.max()

(0.47014984033407031, 0.651685393258427)

In [21]:
get_paper_table(slice_i, f1_choices[1], ['model_name', 'feature_selection', 'upsample'])

test_f1_macro                          RoadNet+Spatial  Social total+Spatial  \
model_name feature_selection upsample                                          
BAGcls     mrmr              regular               NaN              0.316688   
                             svm                   NaN              0.271221   
           rfecv_linsvc      None             0.412796              0.274636   
                             regular          0.385916              0.289890   
                             svm              0.429230              0.302699   
XGBcls     mrmr              regular               NaN              0.341128   
                             svm                   NaN              0.322810   
           rfecv_linsvc      None             0.355515              0.299827   
                             regular          0.391408              0.333311   
                             svm              0.401133              0.316554   

test_f1_macro                          Social type+Spatial  \
model_name feature_selection upsample                        
BAGcls     mrmr              regular              0.334762   
                             svm                  0.416814   
           rfecv_linsvc      None                 0.370729   
                             regular              0.386152   
                             svm                  0.404882   
XGBcls     mrmr              regular              0.347077   
                             svm                  0.422564   
           rfecv_linsvc      None                 0.411420   
                             regular              0.428370   
                             svm                  0.377886   

test_f1_macro                          RoadNet+Social total+Spatial  \
model_name feature_selection upsample                                 
BAGcls     mrmr              regular                       0.470150   
                             svm                                NaN   
           rfecv_linsvc      None                          0.359424   
                             regular                       0.414540   
                             svm                           0.427440   
XGBcls     mrmr              regular                       0.433200   
                             svm                                NaN   
           rfecv_linsvc      None                          0.342583   
                             regular                       0.399174   
                             svm                           0.408609   

test_f1_macro                          RoadNet+Social type+Spatial  
model_name feature_selection upsample                               
BAGcls     mrmr              regular                           NaN  
                             svm                          0.362994  
           rfecv_linsvc      None                         0.386459  
                             regular                      0.368875  
                             svm                          0.390868  
XGBcls     mrmr              regular                           NaN  
                             svm                          0.465524  
           rfecv_linsvc      None                         0.405899  
                             regular                      0.406669  
                             svm                          0.407594

In [22]:
get_paper_table(slice_i, f1_choices[2], ['model_name', 'feature_selection', 'upsample'])

test_f1_micro                          RoadNet+Spatial  Social total+Spatial  \
model_name feature_selection upsample                                          
BAGcls     mrmr              regular               NaN              0.501124   
                             svm                   NaN              0.501124   
           rfecv_linsvc      None             0.591292              0.480899   
                             regular          0.615730              0.498127   
                             svm              0.636704              0.544944   
XGBcls     mrmr              regular               NaN              0.503371   
                             svm                   NaN              0.519101   
           rfecv_linsvc      None             0.574438              0.517978   
                             regular          0.604494              0.524345   
                             svm              0.606742              0.547753   

test_f1_micro                          Social type+Spatial  \
model_name feature_selection upsample                        
BAGcls     mrmr              regular              0.606742   
                             svm                  0.631461   
           rfecv_linsvc      None                 0.585393   
                             regular              0.604494   
                             svm                  0.611236   
XGBcls     mrmr              regular              0.651685   
                             svm                  0.631461   
           rfecv_linsvc      None                 0.571910   
                             regular              0.608989   
                             svm                  0.606742   

test_f1_micro                          RoadNet+Social total+Spatial  \
model_name feature_selection upsample                                 
BAGcls     mrmr              regular                       0.617978   
                             svm                                NaN   
           rfecv_linsvc      None                          0.596754   
                             regular                       0.620225   
                             svm                           0.633708   
XGBcls     mrmr              regular                       0.629213   
                             svm                                NaN   
           rfecv_linsvc      None                          0.591760   
                             regular                       0.633708   
                             svm                           0.626966   

test_f1_micro                          RoadNet+Social type+Spatial  
model_name feature_selection upsample                               
BAGcls     mrmr              regular                           NaN  
                             svm                          0.629213  
           rfecv_linsvc      None                         0.591011  
                             regular                      0.595506  
                             svm                          0.631461  
XGBcls     mrmr              regular                           NaN  
                             svm                          0.651685  
           rfecv_linsvc      None                         0.607865  
                             regular                      0.640449  
                             svm                          0.644944